In [1]:
import geopandas as gpd

In [2]:
las = gpd.GeoDataFrame.from_file("data/raw/Local_Authority_Districts_May_2024_Boundaries_UK_BGC_-7565825659957132419.geojson")

In [3]:
bmdc = las[las.LAD24CD == "E08000032"].to_crs(crs=4326)

In [4]:
bmdc

,FID,LAD24CD,LAD24NM,LAD24NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry
257,258,E08000032,Bradford,,408395,438626,-1.87389,53.84382,c51e1565-8d09-441d-a315-da484d6b8b47,"POLYGON ((-1.72721 53.91015, -1.72722 53.90896..."


In [5]:
bmdc.to_file("data/raw/bd_lad.geojson")

In [6]:
import petl as etl
from pathlib import Path

In [10]:
DATA_PATH = Path('data/raw/')

In [16]:
bd25 = etl.fromcsv(
    DATA_PATH / "bd25_geo.csv"
).split('PCDS', r'\s+', ['OUT', 'IN'])


In [17]:
bd25

OSLAUA,OSWARD,LAT,LONG,OSEAST1M,OSNRTH1M,OUT,IN
E08000032,E05001347,53.796021,-1.750599,416526,0433330,BD1,1AA
E08000032,E05001347,53.79575,-1.750965,416502,0433300,BD1,1AE
E08000032,E05001347,53.794994,-1.750301,416546,0433216,BD1,1AF
E08000032,E05001347,53.793495,-1.753258,416352,0433047,BD1,1AG
E08000032,E05001347,53.794476,-1.749184,416620,0433157,BD1,1AH


In [52]:
def reducer(k, rows):
  all = list(rows);
  return (
    k,
    len([r for r in all if r.OSLAUA == "E08000032"]),
    len([r for r in all if r.OSLAUA != "E08000032"])
  )

counts = bd25.rowreduce(
  'OUT', reducer,
  header=["postcode", "inside", "outside"]
)

In [53]:
counts.tocsv(DATA_PATH / 'postcode_stats.csv')

In [54]:
counts

postcode,inside,outside
BD1,1578,1
BD10,893,17
BD11,7,365
BD12,700,43
BD13,1137,18


In [ ]:
# https://geoportal.statistics.gov.uk/

In [55]:
wards = gpd.GeoDataFrame.from_file("data/raw/Wards_May_2024_Boundaries_UK_BSC_-6022693383694477628.geojson")

In [56]:
wards

,FID,WD24CD,WD24NM,WD24NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry
0,1,E05000932,Ainsdale,,330510,412099,-3.05155,53.60084,1eb9b5eb-4e02-4cd6-9369-590898e50449,"POLYGON ((333004.896 413223.595, 331790.201 41..."
1,2,E05000933,Birkdale,,333046,414534,-3.01375,53.62305,f2bd4869-85cd-4e75-a9a2-eca173f66a88,"POLYGON ((334138.402 414944.403, 333004.896 41..."
2,3,E05000934,Blundellsands,,330806,399928,-3.04438,53.49150,09fa1a69-5e52-4e5e-b60b-78580d3103e0,"POLYGON ((332044.396 399981.496, 331218.299 39..."
3,4,E05000935,Cambridge,,334745,419336,-2.98908,53.66642,e45fdd86-48fa-4551-9c0d-6431993be9ff,"POLYGON ((335369.503 421842.261, 336660.45 419..."
4,5,E05000936,Church,,332033,397634,-3.02539,53.47105,8890dbf0-ff3d-4bd7-b6b6-e44395e04e69,"POLYGON ((332791.399 398377.705, 332732.504 39..."
...,...,...,...,...,...,...,...,...,...,...
8391,8392,W05001796,St Arvans,Llanarfan,352206,199469,-2.69284,51.69193,071aff22-dcd9-47bb-98ce-6a0226bdc6b3,"POLYGON ((353870.761 205106.999, 353651.7 2043..."
8392,8393,W05001797,St Kingsmark,Llangynfarch,352792,194843,-2.68374,51.65038,ef2dbf61-d8f8-498f-859d-07faa88160ff,"POLYGON ((352718.177 196050.494, 353192.5 1958..."
8393,8394,W05001798,Town,Y Dref,350812,212580,-2.71486,51.80968,bedaeaf1-89f6-4d7a-b3ec-6c0ef000a807,"POLYGON ((351948.073 213452.162, 351110.3 2126..."
8394,8395,W05001799,West End,West End,347579,187362,-2.75795,51.58266,f0366bb2-fc0a-486d-9de7-535ac5405a7c,"POLYGON ((347034.351 188531.062, 347599.584 18..."
